In [ ]:
import numpy as np

# PARTIE 1 - MODEL #  
## Description du sujet #

![Scheme_building_Koechlin_Thomas.png](Scheme_building_Koechlin_Thomas.)

Les portes et les murs sont en bois, $lambois$ est le coefficient de conductivité, $Csp$ est la capacité spécifique du bois et $mv$ la masse volumique.

In [ ]:
lambois = 0.077 
Csp = 1250
mv = 370 #kg/m3

$Up$ est le coefficient thermique de la porte, en bois, et $Uf$ est le coeffcient thermique de la fenêtre, prise en double vitrage.

In [ ]:
Up = 2.5
Uf = 1.5

On impose la température extérieure et On utilise un controleur dans chaque pièce pour imposer la température intérieure. Il y a également un flux dans chaque pièce lié à la présence de personnes.

## Description des paramètres #
$Text$ est la température à l'extérieur et $Tc$ est la température imposée à l'intérieur.  
$hconvext$ est le coefficent de convection sur la surface extérieure des murs.  $hconvint$ est le coefficient de convection sur la surface intérieure des murs.

In [ ]:
Text = 5+273
Tc = 20+273
hconvext = 10 #coef de convection à l'extérieur
hconvint = 2 #coef de onvection à l'intéieur

$emext$ est l'épaisseur des murs en contact avec l'extérieur et $emint$ du mur entre les deux pièces.  
$ep$ est l'épaisseur des portes et ef celle des fenêtres.  
$Spf$ est la surface des portes et des fenêtres.  
$Sm1$ est la surface du mur 1.  
$Sm$ est la surface du mur intérieur et du mur 2

In [ ]:
emext = 30*(10**(-3))
emint = 10*(10**(-3))
ep = 10*(10**(-3))
ef = 3*(10**(-3))
Spf = 2
Sm1 = 30-2*Spf
Sm = 30-Spf

Le volume d'air renouvellé par advection est $Va$, et $Vadot$ est le flux d'air.  $rhoair$ est la masse volumique de l'air et $Cair$ est la capacité thermique de l'air.

In [ ]:
Va = 10*10*3
ACH = 1
Va_dot = ACH/3600 * Va
rhoair = 1.3
Cair = 1

## Calcul des Conductances Thermiques #
### Advection #
$G_{advection} = \rho_{air} . C_{air}. \dot{V_{a}}$

In [ ]:
Gad = rhoair*Cair*Va_dot

## Calcul des Conductances Thermiques #
### Convection #
$G_{convection} = h.S$  
$Gconvext1$ est la convection sur la surface extérieure du mur 1.  $Gconvint1$ est la convection sur la surface intérieure du mur 1.  
$Gconvext2$ est la convection sur la surface extérieure du mur 2.  $Gconvint2$ est la convection sur la surface intérieure du mur 2, ainsi que la convection sur les deux surfaces du mur intérieur.

In [ ]:
Gconvext1 = hconvext*Sm1
Gconvint1 = hconvint*Sm1
Gconvext2 = hconvext*Sm
Gconvint2 = hconvint*Sm 

### Conduction #
$G_{conduction} = {{\lambda.S} \over {e}}$  
$Gcondm1$ est la conductivité de la moitié du mur 1, et $Gcondm2$ est la conductivité de la moitié du mur 2.

In [ ]:
Gcondm1 = lambois*Sm1*2/emext
Gcondm2 = lambois*Sm*2/emext

### Controleur #
$G_{controleur} = K$

In [ ]:
Gc = 10**4

### Conductance Totale du mur intérieur #

In [ ]:
Gint = lambois*Spf*2/ep + 1/((2/(Sm*hconvint))+lambois*Sm*2/emint)

### Conductance de la porte et la fenêtre du mur 1

In [ ]:
G1 = Up*Spf+Uf*Spf

### Conductance de la fenetre du mur 2

In [ ]:
G2 = Uf*Spf

## Calcul des Capacités Thermiques #
$C = m.C_{p}$  
$massem1$ est la masse du mur 1 et $massem2$ est la masse du mur 2. $C1$ est la capacité thermique du mur 1 et $C2$ est la capacité thermique du mur 2.

In [ ]:
massem1 = mv*Sm1*emext
massem2 = mv*Sm*emext
C1 = massem1*Csp
C2 = massem2*Csp

## Equations Différentielles Algébriques
$C.\dot{\theta}=-A^T.G.A\theta+A^T.G.b+f$  
Incidence matrix A

In [ ]:
A = np.zeros((13,8))
A[0,0]=1
A[1,0]=-1
A[1,1]=1
A[2,1]=-1
A[2,2]=1
A[3,2]=-1
A[3,3]=1
A[4,3]=1
A[5,3]=-1
A[5,4]=1
A[6,4]=1
A[6,5]=-1
A[7,5]=1
A[7,6]=-1
A[8,6]=1
A[8,7]=-1
A[9,7]=1
A[10,4]=1
A[11,3]=1
A[12,4]=1
print(A)

Conductance matrix G

In [ ]:
G=np.diag([Gconvext1,Gcondm1,Gcondm1,Gconvint1,G1,Gint,Gconvint2,Gcondm2,Gcondm2,Gconvext2,G2,Gc,Gc])
print(G)

Capacity matrix C

In [ ]:
C = np.zeros((8,8))
C[1,1]=C1
C[6,6]=C2
print(C)

Inputs - Temperature sources b

In [ ]:
B = np.zeros(13)
B[0]=Text
B[4]=Text
B[9]=Text
B[10]=Text
B[11]=Tc
B[12]=Tc
print(B)

Inputs - Flow rate sources f

In [ ]:
f[0]=phi1
f[3]=Q1
f[4]=Q2
f[7]=phi2
print(f)

# PARTIE 2 - REGIME PERMANENT #  
## Systemes d'Equations différentielles #
En régime permanent, on a $\dot{\theta}=0$, on résout alors $0=-A^T.G.A\theta+A^T.G.b+f$  


In [ ]:
AT = np.transpose(A)
INV = np.linalg.inv(np.dot(np.dot(AT,G),A))
ATGB = np.dot(np.dot(AT,G),B)
theta0 = np.dot(INV,ATGB+f)
print(theta0-273)

## Représentation d'état #
On utilise la représentation d'état, on souhaite extraire les valeurs de la température aux noeuds 3 et 4 (dans les deux pièces)

In [ ]:
Y = np.zeros(8)
Y[3]=1
Y[4]=1
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, B, C, f, Y)

On résout alors :

In [ ]:
theta = -np.dot(np.dot(np.linalg.inv(As),Bs),U)
print(theta-273)

On remarque que les solutions obtenues avec le systeme différentiel et avec la représentation d'état sont les mêmes.

# PARTIE 3 - MODELE DYNAMIQUE #  